In [ ]:
%load_ext autoreload
%autoreload
from fedhybrid_innerloops import FedHybrid
from tune_and_draw import TuneAndDraw
from parameter import TuneParam,TrainParam
from costFunc import QuadraticCostFunc
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import random

from sklearn.datasets import load_boston
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

import json
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.optimize import minimize

In [ ]:
def dataset(nclient):
    np.random.seed(2022)
    random.seed(2022)
    boston_dataset = load_boston()
    boston = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
    boston['MEDV'] = boston_dataset.target
    X = boston.drop('MEDV', axis = 1) #Drop columns
    y = boston['MEDV']  
    scaler = preprocessing.StandardScaler()
    X = scaler.fit_transform(X)#x-mean/std(Standard deviation)
    X = pd.DataFrame(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 5)#Pass an int for reproducible output across multiple function calls.
    # Not uniformly distributed among clients
    X_train, y_train = shuffle(X_train, y_train)
    #for python array we do not need to use copy function but for numpy array, we need to use copy function
    X_df = X_train.copy()
    X_df['y_train'] = y_train
    
    X_df = X_df.reset_index(drop=True)

    X_df_sorted = X_df.sort_values(by = ['y_train'])
    
    X_df_sorted = X_df_sorted.reset_index()
    #print(X_df_sorted.head(5))# save original index and add new index
    X_train = X_df.drop('y_train', axis = 1)
    y_train = X_df['y_train']
    n = len(y_train)
    print('n',n)
    ndim = X_train.shape[1] + 1  # to add vector of all ones
    print(ndim)
    # number of samples 
    #Compared to the original X_train, it has a new index. start from 0 
    #print('X_train.shape[1])',X_train.shape[1])
    #print('1: ',X_train.head(5))
    intercept = np.ones((X_train.shape[0], 1))
    #print(intercept)
    X_train = np.hstack((intercept, X_train)) #Stack arrays in sequence horizontally (column wise).
    #print('2: ',X_train[0])
    intercept = np.ones((X_test.shape[0], 1))
    X_test = np.hstack((intercept, X_test))
    # number of samples at each client
    p = np.zeros(nclient)
    s = 0
    for i in range(nclient):
        p[i] = random.uniform(0.1, 1)
        s += p[i]
    p = p/s
    ni = p.copy() * n #n=len(y_train)
    ni = [int(num) for num in ni]
    ni[-1] = ni[-1] + n - sum(ni)
    #print(ni)
    indices_set = [[] for i in range(nclient)]
    s = 0
    for j in range(nclient):
        indices_set[j] =  X_df_sorted['index'].to_list()[s:s+ni[j]]
        s += ni[j]
    A = [[] for _ in range(nclient)]
    y = [[] for _ in range(nclient)]
    condiNum = np.zeros(nclient)
    for i in range(nclient):
        A[i] = X_train[indices_set[i]]
        y[i] = y_train[indices_set[i]]
        y[i]=y[i].to_numpy()

    return A,y

In [ ]:
A,y = dataset(8)
condNum= np.zeros(8)
for i in range(8):
    eigenvalue =np.linalg.eigh(np.transpose(A[i]).dot(A[i]))[0]
    condNum[i] = np.sqrt(eigenvalue[-1]/eigenvalue[0])
    if np.isnan(condNum[i]):
        condNum[i] = np.inf
print(condNum)

In [ ]:

def prepare(nclient):
    A,y= dataset(nclient)
    ndim = len(A[0][0])
    func = QuadraticCostFunc(A, y, gamma=1) #cost function 
    initial_x =  np.zeros([ndim,1])
    fn_min = minimize(func.global_func, initial_x, tol=1e-30) #result object
    fn_star = fn_min.fun 
    tuneAndDraw = TuneAndDraw(nclient, initial_x,setup=2)
    method = FedHybrid(func, fn_star)
    return method,tuneAndDraw

synchronous:

In [ ]:
method,tuneAndDraw = prepare(8)
ns=[0,2,4,6,8]
innerIterations = 1
df = tuneAndDraw.readPa(ns,innerIterations)
filename="./analyze data/analyze data/sync_and_async/synchronous quadratic housing.pdf"
tuneAndDraw.draw(df,method.train_innerloop,"synchronous quadratic housing",filename,save = 0,asyn=0,possionBeta=[])

async:

In [ ]:

method,tuneAndDraw = prepare(8)
ns=[0,2,4,6,8]
innerIterations = 0 #only to point to the set of stepsizes with no real meaning 
df = tuneAndDraw.readPa(ns,innerIterations)
filename="./sync_and_async/2 asynchronous quadratic housing3.pdf"
tuneAndDraw.draw(df,method.asynchronousTrain2,"async quadratic housing",filename,save=0,asyn=1,possionBeta = [1,2,3,1,20,25,4,1])

async: to show the bottleneck effect

In [ ]:
method,tuneAndDraw = prepare(8)
ns=[60,15,37,47,25,27,12]
innerIterations = -1 #only to point to the set of stepsizes with no real meaning 
df = tuneAndDraw.readPa(ns,innerIterations)
filename="./sync_and_async/2 async quadratic housing with two Newton-type agents.pdf"
tuneAndDraw.draw(df,method.asynchronousTrain2,"async quadratic housing with two Newton-type agents",filename,save=0,asyn=1,possionBeta = [1,2,30,1,2,4,4,1])

innerloop

In [ ]:
method,tuneAndDraw = prepare(8)
ns=[0,2,4,6,8]
innerIterations =4
df = tuneAndDraw.readPa(ns,innerIterations)
filename="./sync_and_async/2 innerloop 4 quadratic housing before2.pdf"
tuneAndDraw.draw(df,method.train_innerloop_lambdaBefore,"quadratic housing innerloop 4",filename,save = 0,asyn=0,possionBeta=[])